In [1]:
# IMDB - 2 layer network - train + validation + test, 20 epochs, 512 batch size (Tensorflow 1.x version)
# notes: 
# - uses keras to fetch the dataset for consistancy with other code samples, data operations and feeding to network done with numpy

import tensorflow as tf
import tensorflow.contrib.layers as layers

from tensorflow.python import debug as tf_debug

tf.reset_default_graph()

#network parameters
n_input = 10000 #input size for a single sample (10000 words)

#hyperparamters
batch_size = 512
eta = 0.001 # learning rate
max_epoch = 20

# 1. get data (using same dataset as keras example)
from keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=n_input)

#pre-process data into tensors
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences),dimension))
    for i, sequence in enumerate(sequences):
        results[i,sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test =  vectorize_sequences(test_data)

y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

y_train = y_train.reshape(len(y_train),1) #reshape for format taken by tf
y_test = y_test.reshape(len(y_test),1) #reshape for format taken by tf

#validation set to use during training
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

# 2. network architecture
def multilayer_perceptron(x):
    #tf.reset_default_graph()
    fc1 = layers.fully_connected(x, 16, activation_fn = tf.nn.relu, scope = 'fc1')
    fc2 = layers.fully_connected(fc1, 16, activation_fn = tf.nn.relu, scope = 'fc2')
    out = layers.fully_connected(fc2, 1, activation_fn = None, scope = 'out') #no tf.Sigmoid activation function as tf.loss implementation expect raw output
    return out

# 3. select optimizer and loss

#input data placeholders
x = tf.placeholder(tf.float32, [None, n_input], name='placeholder_x')
y = tf.placeholder(tf.float32, name='placeholder_y') 

#network model
y_hat = multilayer_perceptron(x)
#loss
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_hat, labels=y))
#optimizer
train = tf.train.RMSPropOptimizer(learning_rate= eta).minimize(loss)

init = tf.global_variables_initializer()

# 4. train / run network
with tf.Session() as sess:
    sess.run(init)
    
    #define accuracy
    prediction = tf.nn.sigmoid(y_hat)
    correct_prediction = tf.equal(tf.round(prediction), y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    for epoch in range(max_epoch):
        epoch_loss = 0.0
        batch_steps = int(len(partial_x_train) / batch_size)
        for i in range(batch_steps):
            batch_x = partial_x_train[i*batch_size:(i+1)*batch_size]
            batch_y = partial_y_train[i*batch_size:(i+1)*batch_size]
            _, c = sess.run([train, loss], feed_dict = {x: batch_x, y: batch_y })
            epoch_loss += c / batch_steps
        validation_accuracy = accuracy.eval( {x: x_val, y: y_val}  )
        train_accuracy = accuracy.eval( {x: partial_x_train, y: partial_y_train}  )
        print('Epoch %02d, Loss = %.6f, acc: %.6f, validation_acc: %.6f' % (epoch+1, epoch_loss, train_accuracy, validation_accuracy))
        
    # 5. test model
    print("test_acc:", accuracy.eval( {x: x_test, y: y_test}  ))


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 01, Loss = 0.694392, acc: 0.502800, validation_acc: 0.489500
Epoch 02, Loss = 0.692409, acc: 0.543800, validation_acc: 0.525300
Epoch 03, Loss = 0.679538, acc: 0.678267, validation_acc: 0.667300
Epoch 04, Loss = 0.576197, acc: 0.846733, validation_acc: 0.828900
Epoch 05, Loss = 0.418976, acc: 0.896467, validation_acc: 0.867300
Epoch 06, Loss = 0.309507, acc: 0.925467, validation_acc: 0.884700
Epoch 07, Loss = 0.236928, acc: 0.943000, validation_acc: 0.887200
Epoch 08, Loss = 0.192487, acc: 0.949733, validation_acc: 0.880900
Epoch 09, Loss = 0.159984, acc: 0.966333, validation_acc: 0.886900
Epoch 10, Loss = 0.129348, acc: 0.969133, validation_acc: 0.879700
Epoch 11, Loss = 0.111089, acc: 0.977800, validation_acc: 0.883700
Epoch 12, Loss = 0.092453, acc: 0.979467, validation_acc: 0.875700
Epoch 13, Loss = 0.078888, acc: 0.985400, validation_acc: 0.880300
Epoch 14,